# Abdulfatah Bahbouh ID:2017003085

## Homework / Assignment

## first, split the train data into features and class label
## initialize and apply knn= 3
## apply both hold out cross-validation and k-fold cross validation with knn
## split the train data into training and testing
## holdout cross validation: 1) 70:30, 2) 80:20, 3) 90:10
### Q1) Which combination (1,3) gives the best scoring="roc_auc" and scoring="neg_log_loss"
## apply k-fold cross validation: 5
## Q2) what is your k-fold result? Does it match with your holdout cross validation? ## Which option of cross validation (1,3) matches the closest?
## Use k-fold cross validation to find the optimal number of k. k starts from 2, 200. 
## Q3) From you results, what is the most optimal number of k?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
# INSTALL RAPIDS OFFLINE (FROM KAGGLE DATASET). TAKES 1 MINUTE :-)
import sys
!cp ../input/rapids/rapids.0.16.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz
sys.path = ["/opt/conda/envs/rapids/lib"] + ["/opt/conda/envs/rapids/lib/python3.6"] + ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import cuml
print('RAPIDS',cuml.__version__)

## IMPORTS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import *
from sklearn.model_selection import *
import matplotlib.pyplot as plt
import cudf,cuml
from cuml.neighbors import KNeighborsClassifier as cuKNeighbors
from cuml..model_selection import cross_val_score as cu_cross_val_score
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import *

# Question 1

# A. 70%, 30% split

## Readying the Data

In [ ]:
train = pd.read_csv("../input/otto-group-product-classification-challenge/train.csv")
test = pd.read_csv("../input/otto-group-product-classification-challenge/test.csv")


print
#spliting the Training set into features and classes(label)
req_data = train.iloc[:, 1:]

shuffle_index = np.random.permutation(req_data.shape[0])
data = req_data.iloc[shuffle_index]



data = req_data.values

le = LabelEncoder()
data_labels = le.fit_transform(train['target'])

    
training_data,testing_data,training_labels,testing_labels = train_test_split(data[:, :-1],data_labels,test_size = 0.3)

## Creating the Classifier

In [ ]:
classifier = cuKNeighbors(n_neighbors = 3)
classifier.fit(training_data,training_labels)

In [ ]:




y_pred = classifier.predict(testing_data)

#cross_val_score(classifier, x_train, y_train, cv=5,scoring = "accuracy")
#from sklearn.metrics import accuracy_score
#accuracy = accuracy_score(y_test,y_pred)

In [ ]:
y_pred

In [ ]:
accuracy = accuracy_score(testing_labels, y_pred)
print('accuracy',accuracy)
neg_log_loss = cross_val_score(classifier, training_data, training_labels, scoring='neg_log_loss').mean()
print('neg_log_loss', neg_log_loss)
roc_auc = cross_val_score(classifier, training_data, training_labels, scoring='roc_auc_ovr').mean()
print('roc_auc', roc_auc)

# B. 80%, 20% split

In [ ]:
training_data,testing_data,training_labels,testing_labels = train_test_split(data[:, :-1],data_labels,test_size = 0.2)
classifier = cuKNeighbors(n_neighbors = 3)
classifier.fit(training_data,training_labels)
y_pred = classifier.predict(testing_data)

accuracy = accuracy_score(testing_labels, y_pred)
print('accuracy',accuracy)
neg_log_loss = cross_val_score(classifier, training_data, training_labels, scoring='neg_log_loss').mean()
print('neg_log_loss', neg_log_loss)
roc_auc = cross_val_score(classifier, training_data, training_labels, scoring='roc_auc_ovr').mean()
print('roc_auc', roc_auc)

# C. 90%, 10% split

In [ ]:
training_data,testing_data,training_labels,testing_labels = train_test_split(data[:, :-1],data_labels,test_size = 0.1)
classifier = cuKNeighbors(n_neighbors = 3)
classifier.fit(training_data,training_labels)
y_pred = classifier.predict(testing_data)

accuracy = accuracy_score(testing_labels, y_pred)
print('accuracy',accuracy)
neg_log_loss = cross_val_score(classifier, training_data, training_labels, scoring='neg_log_loss').mean()
print('neg_log_loss', neg_log_loss)
roc_auc = cross_val_score(classifier, training_data, training_labels, scoring='roc_auc_ovr').mean()
print('roc_auc', roc_auc)

# The best performing split was the 90/10 one in which the results were neg_log_loss -3.727657221856248, roc_auc 0.9059853332429448.

# Question 2. K-fold Cross validation

In [ ]:
training_data,testing_data,training_labels,testing_labels = train_test_split(data[:, :-1],data_labels,test_size = 0.1)
classifier = cuKNeighbors(n_neighbors = 3)
classifier.fit(training_data,training_labels)

accuracy = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='accuracy').mean()
print('accuracy',accuracy)
neg_log_loss = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='neg_log_loss').mean()
print('neg_log_loss', neg_log_loss)
roc_auc = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='roc_auc_ovr').mean()
print('roc_auc', roc_auc)

# The result for the K-fold cross validation were similar to the 90/10 split but slightly better where we achieved neg_log_loss -3.683687008212811, roc_auc 0.9068099502512361.

# Question 3. Finding the best K value between 2, 200.

In [ ]:
Acc_scores = []
NEG_LOG_LOSS = []
ROC_AUC = []

for k in tqdm(range(2,200)):
    classifier = cuKNeighbors(n_neighbors=k)
    accuracy = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='accuracy').mean()
    Acc_scores.append(accuracy)
    neg_log_loss = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='neg_log_loss').mean()
    NEG_LOG_LOSS.append(neg_log_loss)
    roc_auc = cross_val_score(classifier, training_data, training_labels, cv=5, scoring='roc_auc_ovr').mean()
    ROC_AUC.append(roc_auc)

In [ ]:
for idx,val in enumerate(Acc_scores):
    plt.scatter(int(idx + 2), val)

In [ ]:
for idx,val in enumerate(NEG_LOG_LOSS):
    plt.scatter(int(idx + 2), val)

In [ ]:
for idx,val in enumerate(ROC_AUC):
    plt.scatter(int(idx + 2), val)

In [ ]:
 for idx,val in enumerate(Acc_scores):
    print(f'K = {idx + 2} with Accuracy {val}')

# Overall the best K seems to be somewhere around K = 8

In [ ]:
print(f'K = {Acc_scores.index(max(Acc_scores)) + 2} with the best Accuracy = {max(Acc_scores)}') # the 2 is added to compensate for the shifted array which start at 2


We g